In [ ]:
import numpy as np
from scipy.optimize import linprog
from itertools import *

def powerset(iterable):
    "powerset([1,2,3]) → () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [ ]:
delta = 0.001
epsilon = 0.00001

In [ ]:
variables = [i + str(j) for i in "abc" for j in range(1,8)] + ['nu']

for I1, I2, I3 in product(powerset([2, 3, 6, 7]), repeat=3):
    I1 = (1,) + I1
    I2 = (1,) + I2
    I3 = (1,) + I3
    i = "".join(str(j) for j in I1) + ";" + "".join(str(j) for j in I2) + ";" + "".join(str(j) for j in I3)
    variables.append(("m", I1, I2, I3))
    variables.append(("z", I1, I2, I3))

rev = {j:i for i,j in enumerate(variables)}
integrality = np.zeros((len(variables),))
for i, j in rev.items():
    if i[0] == 'z':
        integrality[j] = 1

In [ ]:
def b4p2(size):
    A = np.zeros((3, size))
    b = np.array([1, 1, 1])
    A[0,0:7] = A[1,7:14] = A[2,14:21] = [1, 2, 3, 4, 5, 6, 7]
    return (A, b)

In [ ]:
def b4p3(size):
    A = np.zeros((3, size))
    b = np.array([epsilon - 0.66, epsilon - 0.66, epsilon - 0.66])
    A[:,:21] = -1
    A[0,0:7] = A[1,7:14] = A[2,14:21] = 0
    return (A, b)

In [ ]:
def b4p4(size):
    A = np.zeros((1, size))
    b = np.array([1 + delta - epsilon])
    A[:,:21] = 1
    return (A, b)

In [ ]:
def b4p5(size):
    A = np.zeros((6, size))
    b = np.zeros((6,))
    b[:3] = 0.34 + delta - epsilon / 2
    b[3:] = delta - 0.32
    A[0,0:7] = A[1,7:14] = A[2,14:21] = 1
    A[3,0:7] = A[4,7:14] = A[5,14:21] = -1
    return (A, b)

In [ ]:
def geom(size):
    A = np.zeros((5 * 2 ** (4 * 3), size))
    b = np.zeros((5 * 2 ** (4 * 3),))

    for q, (I1, I2, I3) in enumerate(product(powerset([2, 3, 6, 7]), repeat=3)):
        I1 = (1,) + I1
        I2 = (1,) + I2
        I3 = (1,) + I3
        f = np.zeros((size,))
        for i in I1:
            A[5 * q, i - 1] = 1
            f[i - 1] = i
        for i in I2:
            A[5 * q, i - 1 + 7] = 1
            f[i - 1 + 7] = i
        for i in I3:
            A[5 * q, i - 1 + 14] = 1
            f[i - 1 + 14] = i

        A[5 * q, 21] = 1
        A[5 * q, 2 * q + 22] = -1
        b[5 * q] = delta

        A[5 * q + 1, 2 * q + 22] = 1
        A[5 * q + 1, 2 * q + 23] = -2
        A[5 * q + 1] -= f
        b[5 * q + 1] = 0

        A[5 * q + 2, 2 * q + 22] = 1
        A[5 * q + 2, 2 * q + 23] = 2
        b[5 * q + 2] = 3

        A[5 * q + 3, 2 * q + 22] = -1
        A[5 * q + 3] -= f
        b[5 * q + 3] = 0

        A[5 * q + 4, 2 * q + 22] = -1
        b[5 * q + 4] = -1
    return (A, b)

In [ ]:
def det(size):
    A = np.zeros((216, size))
    b = np.zeros((216,))
    shifts = [(0, 7), (0, 14), (7, 14)]
    val = 0
    for x, y in shifts:
        for p in range(1, 7):
            for q in range(1, 7):
                A[val * 72 + ((p - 1) * 6 + q - 1) * 2, 21] = 1
                A[val * 72 + ((p - 1) * 6 + q - 1) * 2, p - 1 + x] = 1 - 1 / q
                A[val * 72 + ((p - 1) * 6 + q - 1) * 2, q - 1 + y] = 1
                b[val * 72 + ((p - 1) * 6 + q - 1) * 2] = 1 + delta

                A[val * 72 + ((p - 1) * 6 + q - 1) * 2 + 1, 21] = 1
                A[val * 72 + ((p - 1) * 6 + q - 1) * 2 + 1, p - 1 + x] = 1
                A[val * 72 + ((p - 1) * 6 + q - 1) * 2 + 1, q - 1 + y] = 1 - 1 / p
                b[val * 72 + ((p - 1) * 6 + q - 1) * 2 + 1] = 1 + delta
        val += 1
    return(A, b)

In [ ]:
def thue(size):
    A = np.zeros((9, size))
    b = np.full((9,), 1 + delta)

    A[::3, :7] = [0, 1, 0, 1, 0, 1, 0]
    A[::3, 7:14] = [0, 1, 0, 1, 0, 1, 0]
    A[::3, 14:21] = [0, 1, 0, 1, 0, 1, 0]

    A[1::3, :7] = [0, 0, 1, 0, 0, 1, 0]
    A[1::3, 7:14] = [0, 0, 1, 0, 0, 1, 0]
    A[1::3, 14:21] = [0, 0, 1, 0, 0, 1, 0]

    A[2::3, :7] = [0, 0, 0, 0, 1, 0, 0]
    A[2::3, 7:14] = [0, 0, 0, 0, 1, 0, 0]
    A[2::3, 14:21] = [0, 0, 0, 0, 1, 0, 0]

    A[:3, :7] = 0
    A[3:6, 7:14] = 0
    A[6:9, 14:21] = 0

    A[:, 21] = 1
    return (A, b)

In [ ]:
size = len(rev)
A = np.empty((0, size))
b = np.empty((0,))
c = np.zeros((size,))
c[21] = -1

for f in [b4p2, b4p3, b4p4, b4p5, thue, det, geom]:
    A2, b2 = f(size)
    A = np.vstack([A, A2])
    b = np.hstack([b, b2])

In [ ]:
sol = linprog(c, A, b, integrality=integrality)

In [ ]:
sol

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -0.6676666666666665
              x: [ 5.110e-02  6.485e-14 ...  1.333e+00  0.000e+00]
            nit: -1
          lower:  residual: [ 5.110e-02  6.485e-14 ...  1.333e+00
                              0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00 ...  3.040e+00
                              3.333e-01]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
 mip_node_count: 21735
 mip_dual_bound: -0.6676775

In [ ]:
list(sol.x)

[0.05110444444439025,
 6.484812686835759e-14,
 0.1666666666666667,
 0.11222388888887,
 0.0,
 0.0,
 0.0,
 0.051104444444442065,
 -0.0,
 0.16666666666666655,
 0.11222388888888957,
 0.0,
 0.0,
 0.0,
 0.02439777777779591,
 0.04005999999996851,
 0.16666666666665447,
 0.09887055555563737,
 0.0,
 0.0,
 0.0,
 0.6676666666666665,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0000000000000002,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.126606666666616,
 -0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.1666666666665844,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.126606666666616,
 -0.0,
 1.126606666666616,
 0.0,
 1.0,
 1.0,
 1.1666666666665844,
 0.0,
 1.1666666666665844,
 0.0,
 1.0,
 1.0,
 1.126606666666616,
 

In [ ]:
238 + 5 * 2 ** (2 * 3)

558

In [ ]:
A2, b2 = det(150)

In [ ]:
A2.shape

(216, 150)

In [ ]:
A2[215]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])